In [1]:
import pickle
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
df = pd.read_pickle("./data/HR_proper_light.pkl")
print(df.shape)
df.head()

(167783, 17)


,id,userId,gender,sport,heart_rate_mean,heart_rate_std,duration_s,start_end,altitude_mean,altitude_std,ascend_m,descend_m,distance_mean_m,distance_std_m,distance_total_m,speed_mean,speed_std
0,396826535,10921915,male,bike,152.650,10.229736,7589.0,"[1408898746.0, 1408906335.0]",43.0712,13.475900,407.6,413.0,106.309446,57.798961,53048.413644,7.117435,2.182678
1,392337038,10921915,male,bike,147.710,10.970046,4440.0,"[1408221682.0, 1408226122.0]",35.7248,10.376062,225.4,228.8,65.063260,42.196571,32466.566953,7.550103,2.272863
2,389643739,10921915,male,bike,140.554,11.319500,6749.0,"[1407858459.0, 1407865208.0]",62.4256,13.217515,328.8,352.4,90.798738,50.482550,45308.570377,7.154326,2.150951
3,386729739,10921915,male,bike,147.020,11.414885,4519.0,"[1407432042.0, 1407436561.0]",32.6104,11.417463,242.8,238.6,65.188197,41.980981,32528.910062,7.416873,2.201268
4,383186560,10921915,male,bike (transport),167.154,10.414523,1357.0,"[1406909434.0, 1406910791.0]",22.7620,8.383674,89.6,107.4,23.918070,23.000186,11935.117059,8.721936,2.388718


In [3]:
df_meta = pd.read_csv('./data/Meta.csv')
print(df_meta.shape)
df_meta.head()

(962190, 12)


,userId,hydration,sport,ascent,descent,calories,duration,id,distance,weather,gender,start_time
0,10014612,NaN,mountain bike,147.0,137.0,1313.0,3218.35,322483826,18.85551,7.0,male,2014-04-11T14:32:35.000Z
1,10014612,NaN,mountain bike,97.0,84.0,724.0,1728.33,322483840,10.91231,7.0,male,2014-04-10T14:56:01.000Z
2,10014612,NaN,mountain bike,82.0,94.0,424.0,1904.69,322483848,10.76309,6.0,male,2014-04-07T17:18:13.000Z
3,10014612,NaN,mountain bike,263.0,274.0,2732.0,7218.46,322483865,38.31579,1.0,male,2014-03-30T14:01:32.000Z
4,10014612,NaN,mountain bike,132.0,135.0,1301.0,3208.09,322483880,18.71360,1.0,male,2014-03-28T16:06:04.000Z


In [4]:
df_hr = pd.read_csv('./data/HR.csv')
print(df_hr.shape)
df_hr.head()

(253020, 13)


,userId,gender,id,sport,ave_heart_rate,ave_speed,speed,ave_altitude,max_altitude,min_altitude,start_end,ave_longitude,ave_latitude
0,10921915,male,396826535,bike,152.650,26.162158,"[6.8652, 16.4736, 19.1988, 20.4804, 31.3956, 3...",43.0712,"[171, 74.2]","[468, 14.6]","[1408898746, 1408906335]",24.480066,60.182107
1,10921915,male,392337038,bike,147.710,27.218369,"[9.0792, 13.284, 15.9336, 10.9476, 16.1676, 30...",35.7248,"[255, 67.8]","[415, 17.2]","[1408221682, 1408226122]",24.546053,60.179142
2,10921915,male,389643739,bike,140.554,26.050774,"[4.0464, 13.8204, 16.9776, 19.9404, 28.7856, 3...",62.4256,"[160, 94.0]","[461, 33.4]","[1407858459, 1407865208]",24.505033,60.174214
3,10921915,male,386729739,bike,147.020,26.877838,"[7.4988, 15.4008, 17.8668, 21.5712, 24.1416, 3...",32.6104,"[246, 64.0]","[73, 12.4]","[1407432042, 1407436561]",24.544119,60.179844
4,10921915,male,383186560,bike (transport),167.154,NaN,NaN,22.7620,"[260, 37.8]","[355, 9.8]","[1406909434, 1406910791]",24.731128,60.178065


In [5]:
if 'calories' not in df.columns:
    df = df.merge(df_hr[['id','ave_longitude', 'ave_latitude']], on='id', how='left')
    df = df.merge(df_meta[['id','calories','start_time']], on='id', how='left')
df.shape

(167783, 21)

# Remove outliers

In [6]:
# distribution of number of sessions for each user
def remove_outliers(df_list, whisker_lower=3.0, whisker_upper=3.0,log_transform=False):
    '''
    reomve outliers
    return a pd.Series
    '''
    if log_transform:
        df_list = df_list.apply(lambda x: np.log10(x+1.0) if x>0 else None)
    
    Q1 = df_list.quantile(0.25)
    Q3 = df_list.quantile(0.75)
    IQR = Q3 - Q1
    df_list[~((df_list >= (Q1 - whisker_lower * IQR)) & (df_list <= (Q3 + whisker_upper * IQR)))] = None
    if log_transform:
        with np.errstate(invalid='ignore'):
            df_list = np.power(10, df_list)
    return df_list

def remove_outliers_inplace(df, columns, whisker_lower=3.0, whisker_upper=3.0,log_transform=False):
    '''
    remove outliers of each column in place, replace with NaN's
    columns = a list of strings, indicates column names
    '''
    for col in columns:
        if log_transform:
            df_list = df[col].apply(lambda x: np.log10(x+1.0) if x>0 else None)
        else:
            df_list = df[col]
        Q1 = df_list.quantile(0.25)
        Q3 = df_list.quantile(0.75)
        IQR = Q3 - Q1
        df_list[~((df_list >= (Q1 - whisker_lower * IQR)) & (df_list <= (Q3 + whisker_upper * IQR)))] = None
        if log_transform:
            with np.errstate(invalid='ignore'):
                df_list = np.power(10, df_list)
            
        df[col]=df_list
    return

In [7]:
df.describe()

,id,userId,heart_rate_mean,heart_rate_std,duration_s,altitude_mean,altitude_std,ascend_m,descend_m,distance_mean_m,distance_std_m,distance_total_m,speed_mean,speed_std,ave_longitude,ave_latitude,calories
count,1.677830e+05,1.677830e+05,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,1.672850e+05
mean,3.614781e+08,4.565819e+06,138.707878,12.961956,5348.610318,165.374086,20.677079,242.904824,243.717924,52.466178,25.770214,26180.623049,4.962183,1.498460,12.009655,42.973643,1.406099e+04
std,1.571586e+08,3.906677e+06,18.961298,5.467032,3378.931740,340.389576,36.519386,279.463145,280.093445,47.083246,26.550867,23494.539583,2.362771,2.086149,49.691056,20.652263,5.251500e+06
min,2.579360e+05,6.900000e+01,-800.836000,0.000000,32.000000,-9999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-159.997727,-51.418676,-6.034510e+02
25%,2.581375e+08,1.483178e+06,128.440000,9.486946,2950.000000,19.190200,5.434044,84.800000,86.200000,20.033100,8.928070,9996.516958,3.021586,0.521337,-0.368891,39.559130,5.120600e+02
50%,3.539223e+08,3.365457e+06,140.096000,12.376626,4288.000000,58.621600,10.851233,158.400000,159.600000,33.123972,15.154560,16528.861855,4.434192,1.472856,11.050134,50.365076,7.711170e+02
75%,4.787096e+08,6.919744e+06,150.710000,15.795058,6928.500000,179.308400,20.602921,315.400000,315.800000,72.806155,37.082753,36330.271559,7.063530,2.181533,19.767940,55.656019,1.151000e+03
max,6.740080e+08,1.548142e+07,239.000000,890.310600,18000.000000,9639.748000,2386.186128,29787.000000,29616.000000,470.162079,743.211483,234610.877581,80.995310,476.010448,177.334487,69.664539,2.147480e+09


In [8]:
df.heart_rate_mean.min(), df.heart_rate_mean.max()

(-800.836, 239.0)

In [9]:
df.calories.min(), df.calories.max()

(-603.451, 2147480060.0)

In [10]:
df.distance_total_m.min(), df.distance_total_m.max()

(0.0, 234610.87758119788)

In [11]:
df.duration_s.min(), df.duration_s.max()

(32.0, 18000.0)

In [12]:
remove_outliers_inplace(df, ['heart_rate_mean'],log_transform=False)
remove_outliers_inplace(df, ['duration_s','distance_total_m','calories'],log_transform=True)

In [13]:
df.describe()

,id,userId,heart_rate_mean,heart_rate_std,duration_s,altitude_mean,altitude_std,ascend_m,descend_m,distance_mean_m,distance_std_m,distance_total_m,speed_mean,speed_std,ave_longitude,ave_latitude,calories
count,1.677830e+05,1.677830e+05,167255.000000,167783.000000,167782.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,167783.000000,166391.000000,167783.000000,167783.000000,167783.000000,167783.000000,166351.000000
mean,3.614781e+08,4.565819e+06,139.140938,12.961956,5349.642006,165.374086,20.677079,242.904824,243.717924,52.466178,25.770214,26399.814716,4.962183,1.498460,12.009655,42.973643,928.128084
std,1.571586e+08,3.906677e+06,17.212306,5.467032,3378.916879,340.389576,36.519386,279.463145,280.093445,47.083246,26.550867,23470.682052,2.362771,2.086149,49.691056,20.652263,626.430364
min,2.579360e+05,6.900000e+01,61.992000,0.000000,256.000000,-9999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,208.702905,0.000000,0.000000,-159.997727,-51.418676,47.000000
25%,2.581375e+08,1.483178e+06,128.576000,9.486946,2951.000000,19.190200,5.434044,84.800000,86.200000,20.033100,8.928070,10029.710135,3.021586,0.521337,-0.368891,39.559130,518.000000
50%,3.539223e+08,3.365457e+06,140.158000,12.376626,4289.000000,58.621600,10.851233,158.400000,159.600000,33.123972,15.154560,16756.794596,4.434192,1.472856,11.050134,50.365076,775.000000
75%,4.787096e+08,6.919744e+06,150.752000,15.795058,6929.750000,179.308400,20.602921,315.400000,315.800000,72.806155,37.082753,36643.459350,7.063530,2.181533,19.767940,55.656019,1155.000000
max,6.740080e+08,1.548142e+07,210.194000,890.310600,18001.000000,9639.748000,2386.186128,29787.000000,29616.000000,470.162079,743.211483,234611.877581,80.995310,476.010448,177.334487,69.664539,11772.000000


In [14]:
sport_top_20 = df.sport.value_counts().nlargest(20)
sport_top_20.index

Index(['bike', 'run', 'mountain bike', 'bike (transport)', 'indoor cycling',
       'walk', 'orienteering', 'cross-country skiing',
       'core stability training', 'fitness walking', 'skate', 'roller skiing',
       'hiking', 'kayaking', 'circuit training', 'weight training', 'rowing',
       'gymnastics', 'soccer', 'downhill skiing'],
      dtype='object')

In [15]:
# enough session for each user
userId = df.userId.value_counts().index
userId_ = userId[(df.userId.value_counts()>=10).values]
len(userId), len(userId_)

(1059, 956)

In [16]:
df =df[ df['sport'].isin(sport_top_20.index)
          & df['userId'].isin(userId_)
          & df['heart_rate_mean'].notnull()
          & df['duration_s'].notnull()
          & df['distance_total_m'].notnull()
          & df['calories'].notnull()]

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163955 entries, 0 to 167782
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                163955 non-null  int64  
 1   userId            163955 non-null  int64  
 2   gender            163955 non-null  object 
 3   sport             163955 non-null  object 
 4   heart_rate_mean   163955 non-null  float64
 5   heart_rate_std    163955 non-null  float64
 6   duration_s        163955 non-null  float64
 7   start_end         163955 non-null  object 
 8   altitude_mean     163955 non-null  float64
 9   altitude_std      163955 non-null  float64
 10  ascend_m          163955 non-null  float64
 11  descend_m         163955 non-null  float64
 12  distance_mean_m   163955 non-null  float64
 13  distance_std_m    163955 non-null  float64
 14  distance_total_m  163955 non-null  float64
 15  speed_mean        163955 non-null  float64
 16  speed_std         16

In [17]:
df.to_pickle("./data/df_proper_cleaned.pkl") 

In [18]:
df_ = pd.read_pickle("./data/df_proper_cleaned.pkl")
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163955 entries, 0 to 167782
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                163955 non-null  int64  
 1   userId            163955 non-null  int64  
 2   gender            163955 non-null  object 
 3   sport             163955 non-null  object 
 4   heart_rate_mean   163955 non-null  float64
 5   heart_rate_std    163955 non-null  float64
 6   duration_s        163955 non-null  float64
 7   start_end         163955 non-null  object 
 8   altitude_mean     163955 non-null  float64
 9   altitude_std      163955 non-null  float64
 10  ascend_m          163955 non-null  float64
 11  descend_m         163955 non-null  float64
 12  distance_mean_m   163955 non-null  float64
 13  distance_std_m    163955 non-null  float64
 14  distance_total_m  163955 non-null  float64
 15  speed_mean        163955 non-null  float64
 16  speed_std         16